<a href="https://colab.research.google.com/github/chanseongparkk/BigDataAnalyst_practise/blob/main/adult.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
cd drive/My Drive/빅데이터분석기사 실기/adult

/content/drive/My Drive/빅데이터분석기사 실기


# 성인 인구조사 소득 예측

- age: 나이
- workclass: 고용 형태
- fnlwgt: 사람의 대표성을 나타내는 가중치(final weight)
- education: 교육 수준
- education.num: 교육 수준 수치
- marital.status: 결혼 상태
- occupation: 업종
- relationship: 가족 관계
- race: 인종
- sex: 성별
- capital.gain: 양도 소득
- capital.loss: 양도 손실
- hours.per.week: 주당 근무 시간
- native.country: 국적
- income: 수익 (예측해야 하는 값)

In [ ]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("adult.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='income', null_name='?')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

# 타겟변수 라벨링 변경(0: 50k 이하, 1: 50k 초과)
y_train["income"] = y_train["income"].replace([">50K","<=50K"],[1,0])
y_test["income"] = y_test["income"].replace([">50K","<=50K"],[1,0])

#id열은 분리 후 따로 저장하고 데이터에서 삭제한다.
X_train_id = X_train["id"]
X_test_id = X_test["id"]
y_train_id = y_train["id"]
y_test_id = y_test["id"]
X_train = X_train.iloc[:,1:]
X_test = X_test.iloc[:,1:]
y_train = y_train.iloc[:,1:]
y_test = y_test.iloc[:,1:]

## 전처리
# display(X_train.describe(include = "object"), X_train.describe())
# display(X_test.describe(include = "object"),X_test.describe())

### 결측치 처리

# print(X_train.isna().sum(), X_test.isna().sum())

#수치형 변수는 결측치가 존재하지 않고 범주형 변수는 결측치를 최대 빈도수인 개체로 대체해준다.

X_train["workclass"] = X_train["workclass"].replace(np.nan,X_train["workclass"].value_counts(ascending = False).index[0])
X_train["occupation"] = X_train["occupation"].replace(np.nan,X_train["occupation"].value_counts(ascending = False).index[0])
X_train["native.country"] = X_train["native.country"].replace(np.nan,X_train["native.country"].value_counts(ascending = False).index[0])

X_test["workclass"] = X_test["workclass"].replace(np.nan,X_test["workclass"].value_counts(ascending = False).index[0])
X_test["occupation"] = X_test["occupation"].replace(np.nan,X_test["occupation"].value_counts(ascending = False).index[0])
X_test["native.country"] = X_test["native.country"].replace(np.nan,X_test["native.country"].value_counts(ascending = False).index[0])

# print(X_train.isna().sum(), X_test.isna().sum())

### 변환(로그,제곱근,지수)
#print(X_train.skew(),"\n\n",X_train.kurt())
#print(np.log1p(X_train.iloc[:,[0,2,4,10,11,12]]).skew(),np.log1p(X_train.iloc[:,[0,2,4,10,11,12]]).kurt())
#print(np.sqrt(X_train.iloc[:,[0,2,4,10,11,12]]).skew(),np.sqrt(X_train.iloc[:,[0,2,4,10,11,12]]).kurt())
#print(((X_train.iloc[:,[0,2,4,10,11,12]])**2).skew(),(X_train.iloc[:,[0,2,4,10,11,12]]**2).kurt())
#print(((X_train.iloc[:,[0,2,4,10,11,12]])**3).skew(),(X_train.iloc[:,[0,2,4,10,11,12]]**3).kurt())

X_train.iloc[:,[0,10,11]] = np.log1p(X_train.iloc[:,[0,10,11]])
X_train.iloc[:,2] = np.sqrt(X_train.iloc[:,2])
X_test.iloc[:,[0,10,11]] = np.log1p(X_test.iloc[:,[0,10,11]])
X_test.iloc[:,2] = np.sqrt(X_test.iloc[:,2])
#print(X_train.skew(),"\n\n",X_train.kurt())

### 정규화(StandardScaler)

from sklearn.preprocessing import StandardScaler  

scaler = StandardScaler()

X_train.iloc[:,[0,2,4,10,11,12]] = pd.DataFrame(scaler.fit_transform(X_train.iloc[:,[0,2,4,10,11,12]]), columns = X_train.iloc[:,[0,2,4,10,11,12]].columns,index = X_train.iloc[:,[0,2,4,10,11,12]].index )
X_test.iloc[:,[0,2,4,10,11,12]] = pd.DataFrame(scaler.transform(X_test.iloc[:,[0,2,4,10,11,12]]), columns = X_test.iloc[:,[0,2,4,10,11,12]].columns, index = X_test.iloc[:,[0,2,4,10,11,12]].index)

### 범주형변수 더미화

X_train = pd.get_dummies(X_train,drop_first = True)
X_test = pd.get_dummies(X_test,drop_first = True)

#print(X_train.shape, X_test.shape) # 두 데이터셋의 열 갯수가 다르다

diff = list(set(X_train.columns) - set(X_test.columns)) #차이를 찾는다
diff

X_train = X_train.drop(diff,axis = 1) #test data에 없는 변수를 지워준다.

#print(X_train.shape, X_test.shape) #동일한 형태로 바뀌었다.

### 모델링(RandomForestClassfier)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

rf = RandomForestClassifier(n_estimators = 200, max_depth = 7, min_samples_split = 10, random_state = 999)

rf.fit(X_train, y_train)

print("accuracy : " , rf.score(X_train,y_train), 
      "roc_auc_score : ", roc_auc_score(y_train, rf.predict(X_train)))

### 모델링(XGBClassifier)

from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators = 200, max_depth = 7, subsample = 0.5, random_state = 999)
xgb.fit(X_train,y_train,eval_metric = "auc")
print("accuracy : " , xgb.score(X_train,y_train),
      "roc_auc_score : ", roc_auc_score(y_train, xgb.predict(X_train)))


# XGBoost 모델이 성능이 더 좋아보여 XGBoost 모델을 사용하여 예측값을 구하였다.

pred = xgb.predict(X_test) #예측값
pred_prob = xgb.predict_proba(X_test)[:,0] #0일 확률(50K이하)
final1 = pd.DataFrame({"id" : y_test_id, "pred" : pred})
final2 = pd.DataFrame({"id" : y_test_id, "prob_0" : pred_prob})
# final1.to_csv("0040023351.csv",index=False) #csv생성


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:108: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


accuracy :  0.8518888206388207 roc_auc_score :  0.7311677996027054


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


accuracy :  0.9087070024570024 roc_auc_score :  0.8528934115085158


In [ ]:
pd.read_csv("0040023351.csv")

,id,pred
0,20901,0
1,14170,0
2,1776,1
3,30428,0
4,8602,0
...,...,...
6508,31222,0
6509,10861,0
6510,8929,0
6511,2066,1


In [ ]:
print("accuracy : " , rf.score(X_test,y_test), 
      "roc_auc_score : ", roc_auc_score(y_test, rf.predict(X_test)))
print("accuracy : " , xgb.score(X_test,y_test), 
      "roc_auc_score : ", roc_auc_score(y_test, xgb.predict(X_test)))

accuracy :  0.8493781667434362 roc_auc_score :  0.7237596817161028
accuracy :  0.874251497005988 roc_auc_score :  0.799368168697124
